In [1]:
# prep for notebook
import sys
from pathlib import Path
from pprint import pprint

# 添加项目根目录到路径
notebook_dir = Path().resolve()     # test 目录
project_root = notebook_dir.parent  # 项目根目录
sys.path.insert(0, str(project_root))

import os
os.chdir("E:\\Workshop\\SC-Flint\\sc-cn-desc-generator")

In [13]:
import re
import json
import logging
import yaml

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import src.config as config
from module.replacer import DescTemplateReplacer

with open('config.yaml', 'r', -1, config.ENCODE) as file:
    config_data = yaml.load(file, Loader=yaml.FullLoader)
    project_id = config_data['paratranz']['proj_id']
    auth_token = config_data['paratranz']['token']

In [4]:

'''
stage:
0  - 未翻译
1  - 已翻译
2  - 有疑问
3  - 已检查
5  - 已审核（二校）
9  - 已锁定，此状态下仅管理员可解锁，词条强制按译文导出
-1 - 已隐藏，此状态下词条强制按原文导出
page: 页面
pageSize: 每页大小

返回：
pageCount: 一共几页
'''
url = f"https://paratranz.cn/api/projects/{project_id}/strings?stage=0&pageSize=2"

headers = {'Authorization': auth_token}

response = requests.request("GET", url, headers=headers)
print(response.status_code)
pprint(response.json())

200
{'page': 1,
 'pageCount': 48,
 'pageSize': 2,
 'results': [{'comments': [],
              'context': '',
              'createdAt': '2025-12-11T07:51:47.000Z',
              'file': {'id': 2637743,
                       'name': 'test_paratranz_v0.0.0.json',
                       'project': 17306},
              'fileId': 2637743,
              'history': [],
              'id': 839578686,
              'importHistory': [],
              'key': 'item_DescFood_biscuit_1_a',
              'original': 'NDR: 10\\nEffects: Energizing, '
                          'Hyper-Metabolic\\n\\nThis holiday favorite cookie '
                          'features a crumbly shortbread ring filled with a '
                          'sweet and tart jam. Often enjoyed during Luminalia. '
                          '\\n',
              'project': 17306,
              'stage': 0,
              'translation': 'NDR：10\\n效果：充沛精力，加速代谢\\n\\n这款节日里最受欢迎的曲奇饼干的特点是酥脆的饼圈里填满了酸甜可口的果酱，经常在光灯节期间供人享用。\\n',
              'ui

In [6]:
url = f"https://paratranz.cn/api/projects/{project_id}/strings/{839578692}"

headers = {'Authorization': auth_token}

response = requests.request("GET", url, headers=headers)
print(response.status_code)
pprint(response.json())
print(response.json()['original'])

200
{'comments': [],
 'context': '',
 'createdAt': '2025-12-11T07:51:47.000Z',
 'file': {'id': 2637743,
          'name': 'test_paratranz_v0.0.0.json',
          'project': 17306},
 'fileId': 2637743,
 'history': [],
 'id': 839578692,
 'importHistory': [],
 'key': 'item_Desc_oct_pants_01_Shared',
 'original': 'Carrying Capacity: 1K µSCU\\n\\nOctagon embraces a classic look '
             'and makes it boldly modern with the Altimont. These jodhpur '
             'style pants feature a slightly flared hip that becomes snug '
             'under the knees thanks to built-in garters. A uniquely shaped '
             'front seam secured by silver clips on the waist and shin provide '
             "further detailing that's both subtle and stylish. ",
 'project': 17306,
 'stage': 0,
 'translation': '',
 'uid': None,
 'updatedAt': None,
 'version': 6193182,
 'words': 59}
Carrying Capacity: 1K µSCU\n\nOctagon embraces a classic look and makes it boldly modern with the Altimont. These jodhpur s

In [ ]:
url = f"https://paratranz.cn/api/projects/{project_id}/strings/{839578692}"

headers = {
    'Authorization': auth_token,
    'Content-Type': 'application/json',
}

payload = json.dumps({
   "stage": 0,
   "translation": "测一下API",
})

response = requests.request("PUT", url, headers=headers, data=payload)
print(response.status_code)
pprint(response.json())
print(response.json()['original'])

200
{'context': '',
 'createdAt': '2025-12-11T07:51:47.000Z',
 'fileId': 2637743,
 'history': [],
 'id': 839578692,
 'key': 'item_Desc_oct_pants_01_Shared',
 'original': 'Carrying Capacity: 1K µSCU\\n\\nOctagon embraces a classic look '
             'and makes it boldly modern with the Altimont. These jodhpur '
             'style pants feature a slightly flared hip that becomes snug '
             'under the knees thanks to built-in garters. A uniquely shaped '
             'front seam secured by silver clips on the waist and shin provide '
             "further detailing that's both subtle and stylish. ",
 'project': 17306,
 'stage': 0,
 'translation': '测一下API',
 'uid': 66465,
 'updatedAt': '2025-12-23T04:59:02.845Z',
 'version': 6193182,
 'words': 59}
Carrying Capacity: 1K µSCU\n\nOctagon embraces a classic look and makes it boldly modern with the Altimont. These jodhpur style pants feature a slightly flared hip that becomes snug under the knees thanks to built-in garters. A uniquel

In [ ]:
import logging
import yaml

import src.config as config
from module.paratranz_filler import ParatranzFiller

logging.basicConfig(level=logging.INFO, force=True)

with open('config.yaml', 'r', -1, config.ENCODE) as file:
    config_data = yaml.load(file, Loader=yaml.FullLoader)
    proj_id     = config_data['paratranz']['proj_id']
    auth_token  = config_data['paratranz']['token']

filler = ParatranzFiller(proj_id=proj_id, auth_token=auth_token)
filler.run()

INFO:root:共约100条未翻译条目，开始筛选描述条目
INFO:root:共96条未翻译的描述
INFO:root:开始处理 id: 839578686, key: item_DescFood_biscuit_1_a
INFO:root:ID839578686匹配成功，已填充：【\n（编辑后删除中括号以及中括号以内的内容）\n注意：此为自动填充的模板翻译，文本没有完全翻译且已填充部分可能存在错误，请仔细核对！\n】NDR：10\n效果：精力充沛，快速代谢\n\nThis holiday favorite cookie features a crumbly shortbread ring filled with a sweet and tart jam. Often enjoyed during Luminalia. \n
INFO:root:开始处理 id: 839578687, key: item_DescQDRV_ARCC_S03_Impulse
INFO:root:ID839578687匹配成功，已填充：【\n（编辑后删除中括号以及中括号以内的内容）\n注意：此为自动填充的模板翻译，文本没有完全翻译且已填充部分可能存在错误，请仔细核对！\n】物品类型：Quantum Drive\n制造商：弧光集团 (ArcCorp)\n尺寸：3\n等级：B\n分类：Civilian\n\nLegend claims an inaccurate lathe cut materials thinner than expected on the Impulse prototype. When its test performance exceeded expectations, engineers adopted the changes, and ArcCorp's premiere size three quantum drive was born. 
INFO:root:开始处理 id: 839578690, key: item_DescDrink_bottle_water_old_1_a
INFO:root:ID839578690匹配成功，已填充：【\n（编辑后删除中括号以及中括号以内的内容）\n注意：此为自动填充的模板翻译，文本没有完全翻译且已填充部分可能存在错误，

54